In [23]:
import gradio as gr
from dotenv import load_dotenv
import urllib
import google.generativeai as genai

load_dotenv() # load "/.env" file.

genai.configure() # load GOOGLE_API_KEY from "/.env" file


# function 1 ---------------------------------------------------------------------------------------------------- function 1

#  사진 분석 결과 출력을 위해 입력할 지문
def description() :
    txt = f"사진 속 음식의 이름이 무엇이며 사진 속 음식을 만들기 위해 필요한 재료, 필요한 조리도구, 조리 순서 그리고 재료를 통해 사진 속 음식의 칼로리(kcal)를 예상하여 '음식이름', '재료', '조리도구', '조리순서', '칼로리' 총 5가지 항목으로 나누어 알려주세요."
    txt = f"""
    사진 속 음식에 대해서 음식을 만들기 위해 음식 이름과 필요한 재료, 필요한 조리도구, 조리 순서 그리고 재료를 통해 사진 속 음식의 칼로리(kcal)를 예상하여 다음 형식으로 출력해.
    
    형식:
    ```
    **음식이름** : [음식이름]

    **재료** :
    - [재료 1]
    - [재료 2]

    **조리도구** :
    - [조리도구 1]
    - [조리도구 2]

    **조리순서** :
    1. [조리순서 1]
    2. [조리순서 2]

    **칼로리** :
    [칼로리]
    ```
"""
    return txt

# 문장 추출을 위한 기준 인덱스 선정
def find_index(text, index) :
    text = text[index+2:] # \n 두개 스킵
    to_index = text.find("\n\n")
    return text, to_index

# # 5개 출력
# 음식 이름 추출
def sep_foodName(text, index) :
    # name = text[:index].replace(":","").split() # 1.문단 끝부분까지 string 가져오고, 2.':'있으면 제거하고 공백기준으로 리스트 형태로 분리
    # name = ' '.join(name[1:]) # 리스트->문자열로 변경하기 위해 join(e.g. [비빔, 밥]) / name[0]: 음식이름
    name = text[:index].split(':')[1].strip()
    return name

# 재료 추출
def sep_grad(text, index) :
    # index 및 원하는 2번째 문장 찾기
    text, to_index = find_index(text, index)
    grad_part = text[:to_index] # 문단 끝부분까지 지문 가져오기
    grad = grad_part[grad_part.find("\n")+1:]
    return grad

# 조리도구 추출
def sep_equip(text, index) :
    # index 및 원하는 3번째 문장 찾기
    for i in range(2):
        text, index = find_index(text, index)
    equip_part = text[:index]
    equip = equip_part[equip_part.find("\n")+1:]
    return equip
    
# 조리순서 추출
def sep_recipe(text, index) :
    # index 및 원하는 4번째 문장 찾기
    for i in range(3):
        text, index = find_index(text, index)
    recipe_part = text[:index]
    recipe = recipe_part[recipe_part.find("\n")+1:]
    return recipe

# 칼로리 추출
def sep_kcal(text, index) :
    # index 및 원하는 5번째 문장 찾기
    for i in range(4):
        text, index = find_index(text, index)
    kcal = text[:index].replace(":","").split()
    kcal = ' '.join(kcal[1:])
    return kcal
    
# 프로세스
def process_image(image) :
    # gemnai 모델 불러오기
    vmodel = genai.GenerativeModel('gemini-pro-vision')
    # 프롬프트
    prompt = description()
    response = vmodel.generate_content([prompt, image])
    
    # 초기 문장 범위 설정
    response_text = response.candidates[0].content.parts[0].text
    text = response_text.replace("**", "") # 전체 특수 문자를 공백 대체
    index = text.find("\n\n") # 문단 끝 부분 찾기
    res = sep_foodName(text, index), sep_grad(text, index), sep_equip(text, index), sep_recipe(text, index), sep_kcal(text, index)
    return res



# function 2 ---------------------------------------------------------------------------------------------------- function 2


def recommend_dishes(ingredients):
    prompt = f"주어진 재료들로 어떤 음식을 만들 수 있는지 한 줄로만 콤마(',')로 분리해서 출력해. `{ingredients}`"
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")
        response = model.generate_content(prompt)
        # response = genai.generate_text(prompt=prompt, model="models/text-bison-001")
        # if response.result:
        #     # dishes = response.result.split("\n")  # 음식 리스트를 줄바꿈으로 분리
        #     # print(f"Generated dishes: {dishes}")  # 디버깅을 위한 출력
        #     # return dishes
        response_text = response.candidates[0].content.parts[0].text

        # print(response_text)
        # else:
        #     print("No result from the API")
        #     return []
        # 달걀, 파, 양파, 닭고기
        print( [i.strip() for i in response_text.split(',')])
        return [i.strip() for i in response_text.split(',')]
    except Exception as e:
        print(f"Error in recommend_dishes: {e}")  # 에러 메시지를 출력하여 디버깅에 도움을 줍니다.
        return []

def get_recipe(dish):
    prompt = f"`{dish}`을 어떻게 조리하는지 조리 과정을 설명해."
    try:
        model = genai.GenerativeModel("gemini-1.5-pro")
        response = model.generate_content(prompt)
        if response:
            return response.candidates[0].content.parts[0].text
        else:
            print("No result from the API for the recipe")
            return "레시피를 가져오는 데 오류가 발생했습니다."
    except Exception as e:
        print(f"Error in get_recipe: {e}")  # 에러 메시지를 출력하여 디버깅에 도움을 줍니다.
        return "레시피를 가져오는 데 오류가 발생했습니다."

# ui ---------------------------------------------------------------------------------------------------- ui


# 메인 페이지입니다.
def main_page():
    with gr.Column() as main_view:
        gr.Markdown("## 메인 페이지")
        gr.Markdown("사진으로 음식을 알아보고 조리 과정을 알아보거나, 집에 있는 재료로 어떤 요리를 만들지 알아보세요!")
        cook_page_btn = gr.Button("사진으로 음식 알아보기")
        ingredient_btn = gr.Button("집에 있는 재료로 만들 수 있는 음식은?")
    return main_view, cook_page_btn, ingredient_btn

# process
def process(image, prompt) :
    try:
        vmodel = genai.GenerativeModel('gemini-pro-vision')
        response = vmodel.generate_content([prompt, image])
        return response.text
    except Exception as e:
        return f"Error {e}"

# 조리 제조 과정 페이지. (기능 1)
def cook_page():
    with gr.Column(visible=False) as cook_page:
        # gr.Markdown("## 사진으로 음식을 알아보고 조리에 필요한 준비와 조리 과정을 알아보세요!")
        # gr.Interface(process, [gr.Image(type="pil"), "text"], "text")
        gr.Interface(
            fn=process_image, 
            inputs=[gr.Image(type="pil")], 
            outputs=["text", "text", "text", "text", "text"]
        )
        cook_to_main_btn = gr.Button("메인 페이지로")
    return cook_page, cook_to_main_btn

# 재료로 음식 추천 받기. (기능 2)
def ingredient_page():
    with gr.Column(visible=False) as ingredient_page:
        gr.Markdown("### 음식 재료를 입력하면 만들 수 있는 음식을 추천해드립니다.")

        ingredients = gr.Textbox(label="음식 재료 (콤마로 구분)")
        output_dishes = gr.Dropdown(label="추천된 음식 리스트", choices=[])
        output_recipe = gr.Textbox(label="요리 레시피", interactive=False)

        recommend_button = gr.Button("음식 추천받기")
        get_recipe_button = gr.Button("레시피 확인")

        def update_dishes(ingredients):
            dishes = recommend_dishes(ingredients)
            if dishes:  # 리스트가 비어있지 않다면 업데이트
                output_dishes.choices = dishes
                output_dishes.value = dishes[0] if dishes else ""
            else:  # 비어있다면 초기화
                output_dishes.choices = []
                output_dishes.value = ""
            return gr.Dropdown(choices=dishes, interactive=True)
            
        # 음식 추천 버튼 클릭 시
        recommend_button.click(fn=update_dishes, inputs=ingredients, outputs=output_dishes)
        # 레시피 확인 버튼 클릭 시
        get_recipe_button.click(fn=get_recipe, inputs=output_dishes, outputs=output_recipe)
        ingredient_to_main_btn = gr.Button("메인 페이지로")
    return ingredient_page, ingredient_to_main_btn


# MAIN FUNCITON ---------------------------------------------------------------------------------------------------- MAIN FUNCITON #
def main():
    with gr.Blocks() as app:
        main_view, cook_page_btn, ingredient_btn = main_page()
        cook_view, cook_to_main_btn = cook_page()
        ingredient_view, ingredient_to_main_btn= ingredient_page()

        def gr_update(main_visible, cook_visible, ingredient_visible):
            return gr.update(visible=main_visible), gr.update(visible=cook_visible), gr.update(visible=ingredient_visible)

        def show_main_page():
            return gr_update(True, False, False)
        
        def show_cook_page():
            return gr_update(False, True, False)
    
        def show_ingredient_page():
            return gr_update(False, False, True)
        output = [main_view, cook_view, ingredient_view]

        # button events
        cook_page_btn.click(show_cook_page, [], output)
        ingredient_btn.click(show_ingredient_page, [], output)
        cook_to_main_btn.click(show_main_page, [], output)
        ingredient_to_main_btn.click(show_main_page, [], output)
        

    app.launch()




if __name__ == "__main__":
    main()

Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.
